In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [26]:
X = pd.read_csv("./DeteccionDigitos/digit-recognizer/train.csv", sep=",")
Y = pd.read_csv("./DeteccionDigitos/digit-recognizer/test.csv", sep=",")

sample_submission = pd.read_csv("./DeteccionDigitos/digit-recognizer/sample_submission.csv", sep=",")

In [27]:
X.head(5)
# (42000, 785)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
Y.head(5)
# (28000, 784)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
label = X['label']
label

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, X['label'], test_size=0.2, random_state=42)

Y_train = LabelBinarizer().fit_transform(Y_train)
Y_test = LabelBinarizer().fit_transform(Y_test)

X_train = X_train.drop(columns=['label'])
X_test = X_test.drop(columns=['label'])

print("Train X dataset shape: ", np.shape(X_train))
print("Train y dataset shape: ", np.shape(Y_train))

print("Test X dataset shape: ", np.shape(X_test))
print("Test y dataset shape: ", np.shape(Y_test))

Train X dataset shape:  (33600, 784)
Train y dataset shape:  (33600, 10)
Test X dataset shape:  (8400, 784)
Test y dataset shape:  (8400, 10)


In [31]:
# Define Sequential model with 2 layers
def create_classifier():
    input = Input(shape=(784,))

    x = Dense(100, activation='sigmoid')(input)

    output = Dense(10, activation='softmax')(x)

    return Model(input, output)


model = create_classifier()
model.summary()


model.compile(loss='mse', metrics=['accuracy'], optimizer = Adam())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
history = model.fit(X_train, Y_train, epochs=50, batch_size=500, validation_data=(X_test, Y_test), verbose=1)

Epoch 1/50
68/68 [==============================] - 1s 6ms/step - loss: 0.0474 - accuracy: 0.6858 - val_loss: 0.0264 - val_accuracy: 0.8511
Epoch 2/50
68/68 [==============================] - 0s 4ms/step - loss: 0.0216 - accuracy: 0.8773 - val_loss: 0.0192 - val_accuracy: 0.8852
Epoch 3/50
68/68 [==============================] - 0s 4ms/step - loss: 0.0168 - accuracy: 0.9008 - val_loss: 0.0162 - val_accuracy: 0.9052
Epoch 4/50
68/68 [==============================] - 0s 4ms/step - loss: 0.0147 - accuracy: 0.9137 - val_loss: 0.0147 - val_accuracy: 0.9105
Epoch 5/50
68/68 [==============================] - 0s 4ms/step - loss: 0.0134 - accuracy: 0.9188 - val_loss: 0.0135 - val_accuracy: 0.9185
Epoch 6/50
68/68 [==============================] - 0s 4ms/step - loss: 0.0126 - accuracy: 0.9241 - val_loss: 0.0130 - val_accuracy: 0.9205
Epoch 7/50
68/68 [==============================] - 0s 5ms/step - loss: 0.0119 - accuracy: 0.9282 - val_loss: 0.0124 - val_accuracy: 0.9237
Epoch 8/50
68/68 [==

In [33]:
pred = model.predict(Y)

875/875 [==============================] - 1s 673us/step


In [34]:
df = pd.DataFrame(columns=["ImageId","Label"])

df.Label   = [_.argmax() for _ in pred]
df.ImageId = df.index+1
df.to_csv("./DeteccionDigitos/digit-recognizer/submission.csv", index = False)

df

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,7
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
